In [2]:
# compute
timers = []

import time
def add_timer(note):
  timer = (note, time.time())
  print "timer:", timer
  timers.append(timer)

add_timer('start')

timer: ('start', 1449423117.102322)


In [3]:
import os
from six.moves import urllib

url = 'http://mattmahoney.net/dc/'
def maybe_download(filename, expected_bytes):
  """Download a file if not present, and make sure it's the right size."""
  if not os.path.exists(filename):
    filename, _ = urllib.request.urlretrieve(url + filename, filename)
  statinfo = os.stat(filename)
  if statinfo.st_size == expected_bytes:
    print('Found and verified', filename)
  else:
    print(statinfo.st_size)
    raise Exception(
        'Failed to verify ' + filename + '. Can you get to it with a browser?')
  return filename

In [6]:
# uncomment to download data
# filename = maybe_download('./data/text8.zip', 31344016)
# add_timer('maybe_downloaded')
# print filename


In [7]:
import zipfile

# Read the data into a string.
def read_data(filename):
  f = zipfile.ZipFile(filename)
  for name in f.namelist():
    return f.read(name).split()
  f.close()

    

In [9]:
words = read_data('data/text8.zip')
add_timer('read_data')

timer: ('read_data', 1449423162.37027)


In [10]:
print('Data size', len(words))


('Data size', 17005207)


In [11]:
words[:10]

['anarchism',
 'originated',
 'as',
 'a',
 'term',
 'of',
 'abuse',
 'first',
 'used',
 'against']

In [12]:
vocabulary_size = 50000

In [13]:
import collections

def build_dataset(words):
  count = [['UNK', -1]]
  count.extend(collections.Counter(words).most_common(vocabulary_size - 1))
  dictionary = dict()
  for word, _ in count:
    dictionary[word] = len(dictionary)
  data = list()
  unk_count = 0
  for word in words:
    if word in dictionary:
      index = dictionary[word]
    else:
      index = 0  # dictionary['UNK']
      unk_count = unk_count + 1
    data.append(index)
  count[0][1] = unk_count
  reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
  return data, count, dictionary, reverse_dictionary


In [14]:
data, count, dictionary, reverse_dictionary = build_dataset(words)

In [15]:
add_timer('build_dataset_done')

timer: ('build_dataset_done', 1449423182.974141)


In [16]:
import itertools
list(itertools.islice(dictionary.items(), 5))

[('fawn', 45848),
 ('homomorphism', 9648),
 ('nordisk', 39343),
 ('nunnery', 36075),
 ('chthonic', 33554)]

In [17]:
list(itertools.islice(reverse_dictionary.items(), 5))

[(0, 'UNK'), (1, 'the'), (2, 'of'), (3, 'and'), (4, 'one')]

In [18]:
list(map(lambda x: reverse_dictionary[x], data[:8]))

['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first']

In [19]:
print('Most common words (+UNK)', count[:5])

('Most common words (+UNK)', [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)])


In [20]:
print('Sample data', data[:10])

('Sample data', [5239, 3084, 12, 6, 195, 2, 3137, 46, 59, 156])


In [21]:
import numpy as np
import random

# Step 4: Function to generate a training batch for the skip-gram model.
def generate_batch(batch_size, num_skips, skip_window):
  global data_index
  assert batch_size % num_skips == 0
  assert num_skips <= 2 * skip_window
  random.seed(42)
    
  batch = np.ndarray(shape=(batch_size), dtype=np.int32)
  labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
  span = 2 * skip_window + 1 # [ skip_window target skip_window ]
  buffer = collections.deque(maxlen=span)
  for _ in range(span):
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  for i in range(batch_size // num_skips):
    target = skip_window  # target label at the center of the buffer
    targets_to_avoid = [ skip_window ]
    for j in range(num_skips):
      while target in targets_to_avoid:
        target = random.randint(0, span - 1)
      targets_to_avoid.append(target)
      batch[i * num_skips + j] = buffer[skip_window]
      labels[i * num_skips + j, 0] = buffer[target]
    buffer.append(data[data_index])
    data_index = (data_index + 1) % len(data)
  return batch, labels

In [22]:
data_index = 0
batch, labels = generate_batch(batch_size=8, num_skips=4, skip_window=2)
for i in range(8):
  print(batch[i], '->', labels[i, 0])
  print(reverse_dictionary[batch[i]], '->', reverse_dictionary[labels[i, 0]])


(12, '->', 6)
('as', '->', 'a')
(12, '->', 5239)
('as', '->', 'anarchism')
(12, '->', 3084)
('as', '->', 'originated')
(12, '->', 195)
('as', '->', 'term')
(6, '->', 3084)
('a', '->', 'originated')
(6, '->', 12)
('a', '->', 'as')
(6, '->', 195)
('a', '->', 'term')
(6, '->', 2)
('a', '->', 'of')


In [23]:
import tensorflow as tf


In [24]:
# model hyper parameters
batch_size = 128
embedding_size = 128  # Dimension of the embedding vector.
skip_window = 2       # How many words to consider left and right.
num_skips = 2         # How many times to reuse an input to generate a label.

tf.set_random_seed(42)
num_steps = 100001


In [25]:
# valid_size = 64     # Random set of words to evaluate similarity on.
# valid_window = 300  # Only pick dev samples in the head of the distribution.
# valid_examples = np.array(random.sample(np.arange(valid_window), valid_size))
valid_examples = map(lambda word: dictionary[word], ['often', 'seven', 'people', 'war', 'american', 'history', 'world', 'government', 'music', 'language', 'french', 'church', 'popular', 'science', 'island', 'film'])
num_sampled = 64    # Number of negative examples to sample.

print " ".join(list(map(lambda x: reverse_dictionary[x], valid_examples)))

often seven people war american history world government music language french church popular science island film


In [26]:
import math 

# Compute the average NCE loss for the batch.
# tf.nce_loss automatically draws a new sample of the negative labels each
# time we evaluate the loss.
def nce_loss_fn(inputs_as_embeddings, labels, vocabulary_size, embedding_size, num_sampled):
  # create weights for each word -> embedding, no bias towards any particular word
  nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
  nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

  nce_loss = tf.nn.nce_loss(nce_weights, nce_biases, inputs_as_embeddings, labels, num_sampled, vocabulary_size)
  return tf.reduce_mean(nce_loss)

In [27]:
graph = tf.Graph()
with graph.as_default():
  tf.set_random_seed(42)
  
  # Input data.
  train_inputs = tf.placeholder(tf.int32, shape=[batch_size], name='train_inputs')
  train_labels = tf.placeholder(tf.int32, shape=[batch_size, 1], name='train_labels')
  valid_dataset = tf.constant(valid_examples, dtype=tf.int32, name='valid_dataset')

  # Construct the variables for embeddings
  embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0), name='embeddings')
  input_embeddings = tf.nn.embedding_lookup(embeddings, train_inputs)

  # Define the loss function, and learning rate for descent
  loss = nce_loss_fn(input_embeddings, train_labels, vocabulary_size, embedding_size, num_sampled)
  optimizer = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
  tf.scalar_summary("loss", loss)
  
  # For evaluating progress, compute the cosine similarity between minibatch
  # examples and all embeddings.
  norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keep_dims=True))
  normalized_embeddings = embeddings / norm
  valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
  similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)

In [ ]:
# Step 6: Begin training

# define run filename
import time
ts = time.time()
filename_parts = ["output-basic/", "word2vec_model", "num_steps", str(num_steps), "embedding_size", str(embedding_size), "skip_window", str(skip_window), "_ts", str(ts) +"_"]
base_filename = "_".join(filename_parts)
print "base_filename:", base_filename

add_timer('start_training')
with tf.Session(graph=graph) as session:
  # We must initialize all variables before we use them.
  session.run(tf.initialize_all_variables())
  print("Initialized")
  
  # Add an op that collects all metrics for logging to TensorBoard
  summary_op = tf.merge_all_summaries()
  summary_writer = tf.train.SummaryWriter('summary-logs', session.graph_def)

  average_loss = 0
  for step in xrange(num_steps):
    batch_inputs, batch_labels = generate_batch(
        batch_size, num_skips, skip_window)
    feed_dict = {train_inputs : batch_inputs, train_labels : batch_labels}

    # We perform one update step by evaluating the optimizer op (including it
    # in the list of returned values for session.run()
    _, loss_val = session.run([optimizer, loss], feed_dict=feed_dict)
    average_loss += loss_val

    # log metrics for TensorBoard
    # if step % 100 == 0:
      # summary_str = session.run(summary_op)
      # summary_writer.add_summary(summary_str, step)
      
    # print progress on optimizing for the loss function
    if step % 2000 == 0:
      if step > 0:
        average_loss = average_loss / 2000
      # The average loss is an estimate of the loss over the last 2000 batches.
      print("Average loss at step ", step, ": ", average_loss)
      average_loss = 0

    # and print progress by showing top k similar words for example words
    # note that this is expensive (~20% slowdown if computed every 500 steps)
    if step % 10000 == 0:
      sim = similarity.eval() # TODO(kr)
      for i in xrange(len(valid_examples)):
        valid_word = reverse_dictionary[valid_examples[i]]
        top_k = 8 # number of nearest neighbors
        nearest = (-sim[i, :]).argsort()[1:top_k+1]
        log_str = "Nearest to %s:" % valid_word
        for k in xrange(top_k):
          close_word = reverse_dictionary[nearest[k]]
          log_str = "%s %s," % (log_str, close_word)
        print(log_str)
      
  # evaluate
  final_embeddings = normalized_embeddings.eval()

  # save the variables to disk
  saver = tf.train.Saver()
  save_path = saver.save(session, base_filename + "-tensorflow.ckpt")
  print "Model saved in file: ", save_path
  
add_timer('done_training')

base_filename: output-basic/_word2vec_model_num_steps_100001_embedding_size_128_skip_window_2__ts_1449427134.38_
timer: ('start_training', 1449427134.384286)
Initialized
('Average loss at step ', 0, ': ', 270.50958251953125)
Nearest to often: undisclosed, carbides, yoshi, macs, unconsciousness, gymnastics, svante, repeating,
Nearest to seven: faunas, schiff, bitrate, transshipment, fairey, microscope, waterfalls, merz,
Nearest to people: halloween, madero, ammunition, superceded, melting, palermo, regolith, granger,
Nearest to war: helmut, eyelashes, rss, encumbered, mcgill, whipped, retreat, profusion,
Nearest to american: qualia, diocletian, bmx, rejoin, frege, softness, tombaugh, onlookers,
Nearest to history: standard, standing, maibock, sacra, clamps, stags, zz, peruvian,
Nearest to world: stenosis, transcend, lorry, langevin, collegium, mockery, loomis, templars,
Nearest to government: spirit, turn, direction, dweller, athleticism, recant, abkhazian, bark,
Nearest to music: uc, e

In [29]:
"done"

'done'

In [30]:
final_embeddings.shape


(50000, 128)

In [31]:
final_embeddings[0]


array([ 0.09327951,  0.0675633 ,  0.07638022,  0.01084786,  0.00456135,
       -0.24529585,  0.08709657, -0.0151662 ,  0.01324272,  0.13557342,
        0.00970091,  0.09643302, -0.04931212, -0.01557394,  0.10764762,
       -0.05487974,  0.12147856, -0.03713362, -0.0539389 ,  0.02252143,
       -0.02529177,  0.09988087, -0.09866293, -0.00554492,  0.01920186,
        0.03700102,  0.0675026 ,  0.11687203, -0.01675863,  0.01668186,
        0.05339017, -0.09321076, -0.0034698 ,  0.08625394, -0.11939804,
       -0.04269625,  0.11070847, -0.08010534, -0.08460835, -0.07185863,
        0.01373111, -0.05225307, -0.07576004, -0.05178967,  0.00348074,
       -0.07225302, -0.02150423, -0.00599638, -0.11833259, -0.13213852,
        0.05986469,  0.19564509, -0.07503754, -0.03587047,  0.07478563,
        0.04252718, -0.11255414,  0.02987487,  0.05824735,  0.01388597,
        0.11457191,  0.02706967,  0.03226966, -0.05343361, -0.01878057,
        0.13318062,  0.06418162, -0.04671573,  0.01574231,  0.05

In [32]:
# persist embeddings
final_embeddings_filename = base_filename + "-final_embeddings.npy"
with open(final_embeddings_filename, 'w') as f:
  np.save(f, final_embeddings)

In [33]:
# persist dictionaries
import pickle
with open(base_filename + '-dictionary.pickle', 'w') as f:
  pickle.dump(dictionary, f)
with open(base_filename + '-reversedictionary.pickle', 'w') as f:
  pickle.dump(reverse_dictionary, f)

In [34]:
# persist corpus
# with open(base_filename + '-data.pickle', 'w') as f:
#   pickle.dump(data, f)

In [35]:
add_timer('completely_done')
last_ts = False
for timer in timers:
  note = timer[0]
  ts = timer[1]
  elapsed = ts - last_ts if last_ts != False else 0
  print (str(int(elapsed)).rjust(4) + " seconds", note, ts)
  last_ts = ts
  
print 'total: ' + str(int(timers[-1][1] - timers[0][1])) + ' seconds'
print 'total: ' + str(round((timers[-1][1] - timers[0][1])/60, 2)) + ' minutes'

timer: ('completely_done', 1449424144.70646)
('   0 seconds', 'start', 1449423117.102322)
('  45 seconds', 'read_data', 1449423162.37027)
('  20 seconds', 'build_dataset_done', 1449423182.974141)
('   2 seconds', 'start_training', 1449423185.101875)
(' 958 seconds', 'done_training', 1449424143.334667)
('   1 seconds', 'completely_done', 1449424144.70646)
total: 1027 seconds
total: 17.13 minutes


In [36]:
print "-----------------------------------"
print
print "base_filename:", base_filename
print "final_embeddings_filename:", final_embeddings_filename
print
print "-----------------------------------"
print
print
print "Completely done!"

-----------------------------------

base_filename: word2vec_model_num_steps_100001_embedding_size_128_skip_window_2__ts_1449423185.1_
final_embeddings_filename: word2vec_model_num_steps_100001_embedding_size_128_skip_window_2__ts_1449423185.1_-final_embeddings.npy

-----------------------------------


Completely done!
